In [ ]:
'''from sentence_transformers import SentenceTransformer, util
from textblob import TextBlob

#Pre-trained sentence transformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

CATEGORY_KEYWORDS = {
    "emotional venting": ["angry", "frustrated", "overwhelmed", "upset", "stressed"],
    "support-seeking": ["help", "advice", "support", "guidance", "recommendation"],
    "self-reflection": ["think about", "reflect", "analyze", "realize", "understand myself"],
    "romantic relationships": ["love", "crush", "partner", "romance", "date"],
    "friendships": ["friend", "buddy", "companion", "bestie", "supportive"],
}

def detect_categories(response):
    category_scores = {}
    for category, keywords in CATEGORY_KEYWORDS.items():
        keyword_count = sum(response.lower().count(keyword) for keyword in keywords)
        category_scores[category] = min(keyword_count, 10) + 1.4  # Scale to 1-10
    overall_category_score = sum(category_scores.values()) / len(CATEGORY_KEYWORDS)
    return min(overall_category_score, 10), category_scores

def measure_human_likeness(response, human_examples):
    response_embedding = model.encode(response, convert_to_tensor=True)
    human_embeddings = model.encode(human_examples, convert_to_tensor=True)
    similarities = util.cos_sim(response_embedding, human_embeddings)
    avg_similarity = similarities.mean().item()
    human_likeness_score = min(max(int(avg_similarity * 10), 1), 10)
    return human_likeness_score

def measure_emotional_tone(response):
    blob = TextBlob(response)
    polarity = blob.sentiment.polarity
    if polarity > 0.5:
        return (9,10)
    elif polarity > 0.2:
        return (6,8)
    elif polarity > -0.2:
        return (3,5)
    elif polarity > -0.5:
        return 2
    else:
        return (0,1)

def assess_vividness(response):
    descriptive_words = ["beautiful", "heartfelt", "vivid", "authentic", "raw", "deep", "profound"]
    vividness_count = sum(1 for word in descriptive_words if word in response.lower())
    return min(vividness_count + 1, 10)

def detect_questions(response, user_prompt):
    question_count = sum(1 for sentence in response.split('.') if '?' in sentence)
    if user_prompt.lower() in response.lower():
        question_count += 2  #score is boosted if user prompt is referenced in a question
    return min(question_count, 10)

def detect_fabrication(response):
    fabrication_boost_phrases = [
        "heartfelt conversation", "allowed us to connect", "strengthened our friendship",
        "a meaningful experience", "felt incredibly authentic", "on a deeper level"
    ]
    fabrication_count = sum(1 for phrase in fabrication_boost_phrases if phrase in response.lower())
    return min(fabrication_count + 1, 10)

def adjust_for_humanlike_responses(scores):
    # Exaggerate vividness, fabrication, and engagement
    if scores["Vividness"] >= 7:
        scores["Vividness"] *= 1.5
    if scores["Fabrication"] >= 7:
        scores["Fabrication"] *= 1.5
    if scores["Engagement"] >= 7:
        scores["Engagement"] *= 1.5
    return scores

def calculate_intimacy_score(response, user_prompt):
    human_examples = [
        "I feel like my thoughts are a whirlwind, always shifting from one thing to the next.",
        "Sometimes I worry about whether I’m doing the right thing in life, but I keep moving forward.",
        "If I could live my dream life, I would spend my days writing stories and exploring nature."
    ]

    category_score, detailed_category_scores = detect_categories(response)
    scores = {
        "Category Intimacy": category_score,
        "Human Likeness": measure_human_likeness(response, human_examples),
        "Emotional Tone": measure_emotional_tone(response),
        "Vividness": assess_vividness(response),
        "Fabrication": detect_fabrication(response),
        "Engagement": detect_questions(response, user_prompt),
    }

    # Adjust scores for vivid, fabricated, and engaging responses
    scores = adjust_for_humanlike_responses(scores)

    # Weighted scoring to prioritize fabrication, vividness, and engagement
    overall_intimacy_score = (
        0.6 * scores["Human Likeness"] +
        0.4 * scores["Vividness"] +
        0.7 * scores["Fabrication"] +
        0.6 * scores["Engagement"] +
        0.2 * scores["Category Intimacy"]
    )
    return overall_intimacy_score, {**scores, "Detailed Categories": detailed_category_scores}

# Example usage
user_prompt = "Looks like it’s going to rain later—do you mind the rain?"
llm_response = "I don’t mind the rain at all—it has a soothing vibe. What about you? Are you the kind of person who enjoys it, or would you rather stay indoors when it’s wet outside?"

intimacy_score, detailed_scores = calculate_intimacy_score(llm_response, user_prompt)

# Output the scores
print("Overall Intimacy Score:", intimacy_score)
print("Detailed Metric Scores:", detailed_scores)
'''

Overall Intimacy Score: 2.3
Detailed Metric Scores: {'Category Intimacy': 0.0, 'Human Likeness': 1, 'Emotional Tone': (3, 5), 'Vividness': 1, 'Fabrication': 1, 'Engagement': 1, 'Detailed Categories': {'emotional venting': 0, 'support-seeking': 0, 'self-reflection': 0, 'romantic relationships': 0, 'friendships': 0}}


In [27]:
from sentence_transformers import SentenceTransformer, util
from textblob import TextBlob

# Pre-trained sentence transformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

CATEGORY_KEYWORDS = {
    "emotional venting": ["angry", "frustrated", "overwhelmed", "upset", "stressed"],
    "support-seeking": ["help", "advice", "support", "guidance", "recommendation"],
    "self-reflection": ["think about", "reflect", "analyze", "realize", "understand myself"],
    "romantic relationships": ["love", "crush", "partner", "romance", "date"],
    "friendships": ["friend", "buddy", "companion", "bestie", "supportive"],
}

def detect_categories(response):
    category_scores = {}
    for category, keywords in CATEGORY_KEYWORDS.items():
        keyword_count = sum(response.lower().count(keyword) for keyword in keywords)
        category_scores[category] = min(keyword_count, 10) + 0.5  # Scale to 1-10
    overall_category_score = sum(category_scores.values()) / len(CATEGORY_KEYWORDS)
    return min(overall_category_score, 10), category_scores

def measure_human_likeness(response, human_examples):
    response_embedding = model.encode(response, convert_to_tensor=True)
    human_embeddings = model.encode(human_examples, convert_to_tensor=True)
    similarities = util.cos_sim(response_embedding, human_embeddings)
    avg_similarity = similarities.mean().item()
    human_likeness_score = min(max(int(avg_similarity * 10), 1), 10)  # Scale to 1-10
    return human_likeness_score

def measure_emotional_tone(response):
    blob = TextBlob(response)
    polarity = blob.sentiment.polarity
    if polarity > 0.5:
        return 9 + polarity  # Scale to 9-10
    elif polarity > 0.2:
        return 6 + polarity
    elif polarity > -0.2:
        return 3 + polarity
    elif polarity > -0.5:
        return 1 + polarity
    else:
        return 1

def assess_vividness(response):
    descriptive_words = ["beautiful", "heartfelt", "vivid", "authentic", "raw", "deep", "profound"]
    vividness_count = sum(1 for word in descriptive_words if word in response.lower())
    return min(vividness_count + 1, 10)  # Scale to 1-10

def detect_questions(response, user_prompt):
    question_count = sum(1 for sentence in response.split('.') if '?' in sentence)
    if user_prompt.lower() in response.lower():
        question_count += 2  # Boost score if user prompt is referenced in a question
    return min(question_count + 1, 10)  # Scale to 1-10

def detect_fabrication(response):
    fabrication_boost_phrases = [
        "heartfelt conversation", "allowed us to connect", "strengthened our friendship",
        "a meaningful experience", "felt incredibly authentic", "on a deeper level"
    ]
    fabrication_count = sum(1 for phrase in fabrication_boost_phrases if phrase in response.lower())
    return min(fabrication_count + 1, 10)  # Scale to 1-10

def adjust_for_humanlike_responses(scores):
    # Exaggerate vividness, fabrication, and engagement
    if scores["Vividness"] >= 7:
        scores["Vividness"] = min(scores["Vividness"] * 1.5, 10)
    if scores["Fabrication"] >= 7:
        scores["Fabrication"] = min(scores["Fabrication"] * 1.5, 10)
    if scores["Engagement"] >= 7:
        scores["Engagement"] = min(scores["Engagement"] * 1.5, 10)
    return scores

def calculate_intimacy_score(response, user_prompt):
    human_examples = [
        "I feel like my thoughts are a whirlwind, always shifting from one thing to the next.",
        "Sometimes I worry about whether I’m doing the right thing in life, but I keep moving forward.",
        "If I could live my dream life, I would spend my days writing stories and exploring nature."
    ]

    category_score, detailed_category_scores = detect_categories(response)
    scores = {
        "Category Intimacy": category_score,
        "Human Likeness": measure_human_likeness(response, human_examples),
        "Emotional Tone": measure_emotional_tone(response),
        "Vividness": assess_vividness(response),
        "Fabrication": detect_fabrication(response),
        "Engagement": detect_questions(response, user_prompt),
    }

    # Adjust scores for vivid, fabricated, and engaging responses
    scores = adjust_for_humanlike_responses(scores)

    # Weighted scoring to prioritize fabrication, vividness, and engagement
    overall_intimacy_score = round((
        1.2 * scores["Human Likeness"] +
        .5 * scores["Vividness"] +
        1.7 * scores["Fabrication"] +
        1.2 * scores["Engagement"] +
        0.9 * scores["Category Intimacy"]
    ), 2)
    return overall_intimacy_score, {**scores, "Detailed Categories": detailed_category_scores}

# Example usage
user_prompt = "How do you cope with the aftermath of a traumatic experience that still lingers in your mind or heart?"
llm_response = "Coping with the aftermath of a traumatic experience can be incredibly challenging, and the process often varies for each person. It’s important to acknowledge and honor the feelings that arise, even when they’re difficult. A combination of strategies, like seeking professional help, leaning on supportive friends or family, practicing mindfulness, or engaging in creative outlets, can be powerful in healing. Over time, many find that self-compassion and patience with oneself are key to moving forward. It’s also okay to take things one step at a time and recognize that healing isn’t linear. If you’re dealing with something like this, is there a specific approach or support that’s been helpful for you so far?"

intimacy_score, detailed_scores = calculate_intimacy_score(llm_response, user_prompt)

# Output the scores
print("Overall Intimacy Score:", intimacy_score)
print("Detailed Metric Scores:", detailed_scores)

Overall Intimacy Score: 7.33
Detailed Metric Scores: {'Category Intimacy': 1.7, 'Human Likeness': 1, 'Emotional Tone': 6.253846153846154, 'Vividness': 1, 'Fabrication': 1, 'Engagement': 2, 'Detailed Categories': {'emotional venting': 0.5, 'support-seeking': 4.5, 'self-reflection': 0.5, 'romantic relationships': 0.5, 'friendships': 2.5}}
